# 데이터 사용

데이터는 기계 학습 모델 작성 과정의 토대가 되는 요소라 할 수 있습니다. 전문 데이터 과학 솔루션을 빌드하려면 클라우드에서 중앙 집중식으로 데이터를 관리하고, 여러 워크스테이션과 컴퓨팅 대상에서 실험 실행 및 모델 학습을 진행하는 데이터 과학자 팀에게 해당 데이터 액세스 권한을 제공해야 합니다.

이 Notebook에서는 데이터 사용을 위한 두 가지 Azure Machine Learning 개체인 *데이터 저장소* 및 *데이터 세트*를 살펴봅니다.

## 작업 영역에 연결

이 Notebook의 작업을 시작하려면 먼저 작업 영역에 연결합니다.

> **참고**: Azure 구독에 인증된 세션을 아직 설정하지 않은 경우에는 링크를 클릭하고 인증 코드를 입력한 다음 Azure에 로그인하여 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 데이터 저장소 사용

Azure ML에서 *데이터 저장소*는 Azure Storage Blob 컨테이너 등의 스토리지 위치에 대한 참조입니다. 모든 작업 영역에는 기본 데이터 저장소(대개 작업 영역과 함께 만들어진 Azure Storage Blob 컨테이너)가 있습니다. 다른 위치에 저장된 데이터를 사용해야 하는 경우에는 작업 영역에 사용자 지정 데이터 저장소를 추가하고 그 중 하나를 기본 데이터 저장소로 설정하면 됩니다.

### 데이터 저장소 확인

다음 코드를 사용하여 작업 영역의 데이터 저장소를 확인합니다.

In [ ]:
# 기본 데이터 저장소 가져오기
default_ds = ws.get_default_datastore()

# 모든 데이터 저장소를 열거하고 기본 데이터 저장소 표시
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

[Azure Machine Learning Studio](https://ml.azure.com)의 작업 영역 **데이터 저장소** 페이지에서도 작업 영역의 데이터 저장소를 확인하고 관리할 수 있습니다.

### 데이터 저장소에 데이터 업로드

사용 가능한 데이터 저장소를 확인한 후에는 로컬 파일 시스템에서 데이터 저장소에 파일을 업로드할 수 있습니다. 그러면 실험 스크립트가 실제로 실행되는 위치에 관계없이 작업 영역에서 실행 중인 실험에서 해당 파일에 액세스할 수 있습니다.

In [ ]:
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                       target_path='diabetes-data/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

## 데이터 세트 사용

Azure Machine Learning에서는 *데이터 세트* 형식의 데이터용 추상화가 제공됩니다. 데이터 세트는 실험에서 사용할 수 있는 특정 데이터 세트에 대한 버전이 지정된 참조입니다. 데이터 세트는 *테이블 형식* 또는 *파일* 기반 형식일 수 있습니다.

### 테이블 형식 데이터 세트 만들기

데이터 저장소에 업로드한 당뇨병 데이터에서 데이터 세트를 만들고 첫 20개 레코드를 살펴보겠습니다. 여기서 데이터는 CSV 파일에 포함된 구조적 형식이므로 *테이블 형식* 데이터 세트를 사용합니다.

In [ ]:
from azureml.core import Dataset

# 기본 데이터 저장소 가져오기
default_ds = ws.get_default_datastore()

#데이터 저장소의 경로에서 테이블 형식 데이터 세트 만들기(시간이 다소 걸릴 수 있음)
tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

# 처음 20개 행을 Pandas 데이터 프레임으로 표시
tab_data_set.take(20).to_pandas_dataframe()

위의 코드에 나와 있는 것처럼 테이블 형식 데이터 세트는 Pandas 데이터 프레임으로 쉽게 변환할 수 있습니다. 따라서 일반 Python 기술을 통해 데이터를 사용할 수 있습니다.

### 파일 데이터 세트 만들기

앞에서 만든 데이터 세트는 *테이블 형식* 데이터 세트입니다. 이 데이터 세트는 데이터 세트 정의에 포함된 구조적 파일의 모든 데이터가 들어 있는 데이터 프레임으로 읽을 수 있습니다. 테이블 형식 데이터의 경우 이러한 방식이 적합합니다. 하지만 비구조적 데이터를 사용해야 하는 기계 학습 시나리오도 있고, 직접 작성한 코드에서 파일의 데이터 읽기를 처리하려는 경우도 있을 수 있습니다. 이러한 경우에는 *파일* 데이터 세트를 사용할 수 있습니다. 파일 데이터 세트에서는 가상 탑재 지점에 파일 경로 목록이 작성되는데, 이 목록을 사용하여 파일의 데이터를 읽을 수 있습니다.

In [ ]:
#데이터 저장소의 경로에서 파일 데이터 세트 만들기(시간이 다소 걸릴 수 있음)
file_data_set = Dataset.File.from_files(path=(default_ds, 'diabetes-data/*.csv'))

# 데이터 세트의 파일 가져오기
for file_path in file_data_set.to_path():
    print(file_path)

### 데이터 세트 등록

당뇨병 데이터를 참조하는 데이터 세트를 만든 후에는 작업 영역에서 실행되는 모든 실험에서 쉽게 액세스할 수 있도록 데이터 세트를 등록할 수 있습니다.

테이블 형식 데이터 세트는 **당뇨병 데이터 세트**로 등록하고 파일 데이터 세트는 **당뇨병 파일**로 등록하겠습니다.

In [ ]:
# 테이블 형식 데이터 세트 등록
try:
    tab_data_set = tab_data_set.register(workspace=ws, 
                                        name='diabetes dataset',
                                        description='diabetes data',
                                        tags = {'format':'CSV'},
                                        create_new_version=True)
except Exception as ex:
    print(ex)

# 파일 데이터 세트 등록
try:
    file_data_set = file_data_set.register(workspace=ws,
                                            name='diabetes file dataset',
                                            description='diabetes files',
                                            tags = {'format':'CSV'},
                                            create_new_version=True)
except Exception as ex:
    print(ex)

print('Datasets registered')

[Azure Machine Learning Studio](https://ml.azure.com)의 작업 영역 **데이터 세트** 페이지에서 데이터 세트를 확인하고 관리할 수 있습니다. 작업 영역 개체에서 데이터 세트 목록을 가져올 수도 있습니다.

In [ ]:
print("Datasets:")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name, 'version', dataset.version)

데이터 세트의 버전을 지정하는 기능을 사용하면 이전 정의를 사용하는 파이프라인이나 기존 실험을 그대로 유지하면서 데이터 세트를 다시 정의할 수 있습니다. 기본적으로는 명명된 데이터 세트의 최신 버전이 반환되지만 다음과 같이 버전 번호를 지정하여 데이터 세트의 특정 버전을 검색할 수 있습니다.

```python
dataset_v1 = Dataset.get_by_name(ws, 'diabetes dataset', version = 1)
```


### 테이블 형식 데이터 세트에서 모델 학습 진행

데이터 세트를 만들었으므로 이제 해당 데이터 세트에서 모델 학습을 시작할 수 있습니다. 스크립트를 실행하는 데 사용되는 추정기의 *입력*으로 데이터 세트를 스크립트에 전달할 수 있습니다.

아래의 두 코드 셀을 실행하여 다음 항목을 만듭니다.

1. **diabetes_training_from_tab_dataset** 폴더
2. 분류 모델을 학습시키는 스크립트. 이 스크립트는 전달된 테이블 형식 데이터 세트를 인수로 사용합니다.

In [ ]:
import os

# 실험 파일용 폴더 만들기
experiment_folder = 'diabetes_training_from_tab_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 라이브러리 가져오기
import os
import argparse
from azureml.core import Run, Dataset
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 스크립트 인수(정규화 비율 및 학습 데이터 세트 ID) 가져오기
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument("--input-data", type=str, dest='training_dataset_id', help='training dataset')
args = parser.parse_args()

# 정규화 하이퍼 매개 변수 설정(스크립트에 인수로 전달됨)
reg = args.reg_rate

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 학습 데이터 세트 가져오기
print("Loading Data...")
diabetes = run.input_datasets['training_data'].to_pandas_dataframe()

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 로지스틱 회귀 모델 학습
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 출력 폴더에 저장된 메모 파일이 실험 레코드에 자동으로 업로드됨
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

> **참고**: 스크립트에서 데이터 세트는 매개 변수나 인수로 전달됩니다. 테이블 형식 데이터 세트의 경우 이 인수에는 등록된 데이터 세트의 ID가 포함됩니다. 그러므로 다음과 같이 실행 컨텍스트에서 실험의 작업 영역을 가져온 다음 해당 ID를 사용하여 데이터 세트를 가져오는 코드를 스크립트에 작성할 수 있습니다.
>
> ```
> run = Run.get_context()
> ws = run.experiment.workspace
> dataset = Dataset.get_by_id(ws, id=args.training_dataset_id)
> diabetes = dataset.to_pandas_dataframe()
> ```
>
> 그러나 Azure Machine Learning 실행에서는 명명된 데이터 세트를 참조하는 인수를 자동으로 식별하여 실행의 **input_datasets** 컬렉션에 추가합니다. 따라서 데이터 세트 "식별 이름"을 지정해 이 컬렉션에서 데이터 세트를 검색할 수도 있습니다. 식별 이름(잠시 후에 살펴볼 예정)은 실험의 스크립트 실행 구성에 포함된 인수 정의에서 지정됩니다. 위 스크립트에는 이 방식이 사용되었습니다.

이제 학습 데이터 세트용 인수를 정의하여 스크립트를 실험으로 실행할 수 있습니다. 스크립트는 이 데이터 세트를 읽습니다.

> **참고**: **Dataset** 클래스는 **azureml-dataprep** 패키지의 일부 구성 요소를 사용합니다. 이 패키지에는 **to_pandas_dataframe()** 메서드에 사용되는 **pandas** 지원 구성 요소(선택 사항)가 포함되어 있습니다. 그러므로 학습 실험을 실행할 환경에 이 패키지를 포함해야 합니다.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails


# 실험용 Python 환경 만들기
sklearn_env = Environment("sklearn-env")

# 필수 패키지가 설치되어 있는지 확인합니다. scikit-learn, Azure ML defaults 및 Azure ML dataprep 패키지가 필요합니다.
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults','azureml-dataprep[pandas]'])
sklearn_env.python.conda_dependencies = packages

# 학습 데이터 세트 가져오기
diabetes_ds = ws.datasets.get("diabetes dataset")

# 스크립트 구성 만들기
script_config = ScriptRunConfig(source_directory=experiment_folder,
                              script='diabetes_training.py',
                              arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                           '--input-data', diabetes_ds.as_named_input('training_data')], # Reference to dataset
                              environment=sklearn_env) 

# 실험 제출
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

> **참고:** **--input-data** 인수는 데이터 세트를 *명명된 입력*으로 전달합니다. 이 입력에는 데이터 세트의 *식별 이름*이 포함되어 있는데, 스크립트는 이 이름을 사용하여 실험 실행의 **input_datasets** 컬렉션에서 해당 데이터 세트를 읽습니다. **--input-data** 인수의 문자열 값은 실제로는 등록된 데이터 세트의 ID입니다.  `diabetes_ds.id`만 전달할 수도 있습니다. 이 경우 스크립트는 스크립트 인수에서 데이터 세트 ID에 액세스한 다음 해당 ID를 사용하여 **input_datasets** 컬렉션이 아닌 작업 영역에서 데이터 세트를 가져올 수 있습니다.

실험을 처음 실행할 때는 Python 환경을 설정하는 데 시간이 다소 걸릴 수 있습니다. 후속 실행은 더 빠르게 진행됩니다.

실험이 완료되면 위젯에서 **azureml-logs/70_driver_log.txt** 출력 로그와 실행에서 생성된 메트릭을 확인합니다.

### 학습된 모델 등록

학습된 모델은 모든 학습 실험과 마찬가지로 검색하여 Azure Machine Learning 작업 영역에 등록할 수 있습니다.

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Tabular dataset'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

### 파일 데이터 세트에서 모델 학습 진행

앞에서는 *테이블 형식* 데이터 세트의 학습 데이터를 사용해 모델을 학습시키는 방법을 살펴보았습니다. 이번에는 *파일* 데이터 세트를 사용하는 방법을 알아보겠습니다.

파일 데이터 세트를 사용할 때 스크립트에 전달되는 데이터 세트 인수는 파일 경로가 포함된 탑재 지점을 나타냅니다. 이러한 파일에서 데이터를 읽는 방법은 파일의 데이터 종류와 해당 데이터를 사용하여 수행하려는 작업에 따라 다릅니다. 당뇨병 CSV 파일의 경우에는 Python **glob** 모듈을 사용하여 데이터 세트에 의해 정의된 가상 탑재 지점에서 파일 목록을 만들 수 있으며, 모든 파일을 단일 데이터 프레임으로 연결된 Pandas 데이터 프레임으로 읽어들일 수 있습니다.

아래의 두 코드 셀을 실행하여 다음 항목을 만듭니다.

1. **diabetes_training_from_file_dataset** 폴더
2. 분류 모델을 학습시키는 스크립트. 이 스크립트는 전달된 파일 데이터 세트를 *입력*으로 사용합니다.

In [ ]:
import os

# 실험 파일용 폴더 만들기
experiment_folder = 'diabetes_training_from_file_dataset'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# 라이브러리 가져오기
import os
import argparse
from azureml.core import Dataset, Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import glob

# 스크립트 인수(정규화 비율 및 파일 데이터 세트 탑재 지점) 가져오기
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument('--input-data', type=str, dest='dataset_folder', help='data mount point')
args = parser.parse_args()

# 정규화 하이퍼 매개 변수 설정(스크립트에 인수로 전달됨)
reg = args.reg_rate

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 당뇨병 데이터 세트 로드
print("Loading Data...")
data_path = run.input_datasets['training_files'] # Get the training data path from the input
# (하드 코드된 이름을 사용하지 않으려는 경우 args.dataset_folder만 사용해도 됩니다.)

# 파일 읽기
all_files = glob.glob(data_path + "/*.csv")
diabetes = pd.concat((pd.read_csv(f) for f in all_files), sort=False)

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 로지스틱 회귀 모델 학습
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 출력 폴더에 저장된 메모 파일이 실험 레코드에 자동으로 업로드됨
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

파일 데이터 세트도 테이블 형식 데이터 세트와 마찬가지로 식별 이름을 사용해 **input_datasets** 컬렉션에서 검색할 수 있습니다. 스크립트 인수에서 파일 데이터 세트를 검색할 수도 있습니다. 테이블 형식 데이터 세트의 경우 이 인수에 전달된 데이터 세트 ID가 포함되는 반면, 파일 데이터 세트의 경우에는 이 인수에 파일의 탑재 경로가 포함됩니다.

다음으로는 스크립트에 데이터 세트를 전달하는 방식을 변경해야 합니다. 즉, 스크립트가 파일을 읽을 수 있는 경로를 정의해야 합니다. **as_download** 또는 **as_mount** 메서드를 사용하여 이 변경을 수행할 수 있습니다. **as_download** 메서드를 사용하면 파일 데이터 세트의 파일이 스크립트를 실행하는 컴퓨팅 대상의 임시 위치에 다운로드됩니다. 반면 **as_mount** 사용 시에는 데이터 저장소에서 파일을 직접 스트리밍할 수 있는 탑재 지점이 생성됩니다.

액세스 방법과 **as_named_input** 메서드를 함께 사용하면 실험 실행의 **input_datasets** 컬렉션에 데이터 세트를 포함할 수 있습니다. 인수를 `diabetes_ds.as_mount()` 등으로 설정하여 이 메서드를 생략하면 스크립트가 스크립트 인수에서는 데이터 세트 탑재 지점에 액세스할 수 있지만 **input_datasets** 컬렉션에서는 탑재 지점에 액세스할 수 없습니다.

In [ ]:
from azureml.core import Experiment
from azureml.widgets import RunDetails


# 학습 데이터 세트 가져오기
diabetes_ds = ws.datasets.get("diabetes file dataset")

# 스크립트 구성 만들기
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_training.py',
                                arguments = ['--regularization', 0.1, # Regularizaton rate parameter
                                             '--input-data', diabetes_ds.as_named_input('training_files').as_download()], # Reference to dataset location
                                environment=sklearn_env) # Use the environment created previously

# 실험 제출
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

실험이 완료되면 위젯에서 **azureml-logs/70_driver_log.txt** 출력 로그를 표시하여 스크립트가 파일을 읽을 수 있도록 파일 데이터 세트의 파일이 임시 폴더에 다운로드되었는지 확인합니다.

### 학습된 모델 등록

이번에도 실험을 통해 학습된 모델을 등록할 수 있습니다.

In [ ]:
from azureml.core import Model

run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'File dataset'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **자세한 정보**: 데이터 세트를 사용하는 학습에 대한 자세한 내용은 Azure ML 설명서의 [데이터 세트로 학습](https://docs.microsoft.com/azure/machine-learning/how-to-train-with-datasets)을 참조하세요.